## Data Modeling
***
During the modeling process, I iteratively trained and evaluated numerous models to identify the most effective one. A validation dataset, previously unseen by the model, was used for evaluation to ensure accuracy in real-world scenarios. To inform my decision-making, I relied on both loss and recall metrics, aiming to minimize error while maximizing the model's ability to correctly identify pneumonia cases. I also included the Area Under the ROC Curve (AUC) to provide insight into the model's overall performance, ensuring it doesn't just achieve high recall by simply labeling all cases as positive.

Key adjustments made during model iterations and hyperparameter tuning included:

- Adjusting the number and configuration of layers and nodes.
- Choosing between Dense and Conv2D layers for structure.
- Implementing regularization techniques such as Dropout layers, L1/L2 regularization, and data augmentation to prevent overfitting.
- Selecting appropriate initializers and optimizers for optimization.
- Applying transfer learning with the pretrained ResNet50 model to leverage pre-existing neural network architectures.